# **Analyzing Customer Satisfaction Trends in 2022**



**Introduction:**

This notebook aims to analyze customer satisfaction trends across various regions, products and case owners during first quarter of the year 2022. The provided dataset consists of several attributes, including customer satisfaction scores, product information and case details. The objective of this analysis is to identify factors contributing to a downward trend in the Overall Technician Satisfaction rating starting in January and to provide actionable recommendations to improve these ratings.


**Data Loading, Transformations and Initial Inspection:**

To begin our analysis, we load the customer satisfaction dataset into a Spark DataFrame from AWS S3. We perform some necessary data transformations to standardize certain fields, ensuring consistency in our analysis. For example, country names are unified and the language of the Issue_Resolved field is standardized to English. These transformations will help in the subsequent analysis, particularly in regional and issue resolution impact studies.

After performing these transformations, we print the schema of the DataFrame to understand the data types of each column. Finally, the first 10 rows of the dataset are displayed to inspect the data structure, giving us an overview of the key attributes available for our analysis.

In [0]:
# Import data from s3 AWS
df = spark.read.option("sep", ",") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("s3n://humber-lfb-databricks-class-files/midterm_helpdesk.csv")
# "sep" is used to specify the delimiter of CSV file, "header" is used to specify the first row is a header, and "inferSchema" is used to automatically infer the schema

# Display the updated DataFrame
display(df)

Client_ID,Main_Product,Number_Users,Case_Number,Overall_Technician_Satisfaction,Recommend_Friend_Colleague,Issue_Resolved,First_Time_Support_Contacted,Created_Date,Case_Owner,Category,Sub_Category,Support_Tier,City_state,Country,Billing_Country
FARME12,Epic Cloud,29,3841693,4,4,No,Yes,2022-01-02,Angelica Garcia,Epic,Download & RT Interface,Gold Advanced,"Denver, CO",United States,United States
ONECA01,Epic Cloud,39,3973271,8,8,Yes,Yes,2022-03-11,Jamar Carpenter,TAM,General,Gold Advanced,"Cedarhurst, NY",United States,United States
EMERA02,TAM Cloud,5,3846734,10,10,Yes,Yes,2022-01-03,Joshua Gralewski,TAM,General,Gold Intermediate,"Emerald Isle, NC",null,United States
NFPCA01,Epic Cloud,960,3972845,10,10,Yes,Yes,2022-03-11,David Osmolski,Epic,General,Platinum,"Ancaster, ON",United States,Canada
AZALE01,TAM Cloud,12,3842761,10,10,Yes,Yes,2022-01-03,Jamie Mercer,TAM,General,Gold Intermediate,"Wilmington, NC",null,United States
PALL-01,Epic Cloud,55,3973280,10,10,Yes,Yes,2022-03-11,Aric Calabrese,CSR24,CSR24,Gold Advanced,"Orleans, ON",null,Canada
REEDW02,Epic Cloud,43,3846783,10,10,Yes,Yes,2022-01-03,Jessica Hale,TAM,General,Gold Advanced,"Bedford, PA",United States,United States
BRYSO02,Epic Cloud,104,3973321,9,9,Yes,Yes,2022-03-11,Tammy Martin,Epic,Accounting,Platinum,"Ajax, ON",Canada,Canada
HARBI01,Epic Cloud,24,3846791,10,10,Yes,Yes,2022-01-03,Allison Arseneau,Epic,Accounting,Gold Intermediate,"Tyrone, GA",United States,United States
GRANI04,Epic Cloud,33,3973363,10,10,Yes,Yes,2022-03-11,Shareem Mohamed,Epic,Download & RT Interface,Gold Intermediate,"Granite Falls, NC",null,United States


In [0]:
#To check the schema of the data
df.printSchema()

root
 |-- Client_ID: string (nullable = true)
 |-- Main_Product: string (nullable = true)
 |-- Number_Users: integer (nullable = true)
 |-- Case_Number: integer (nullable = true)
 |-- Overall_Technician_Satisfaction: integer (nullable = true)
 |-- Recommend_Friend_Colleague: integer (nullable = true)
 |-- Issue_Resolved: string (nullable = true)
 |-- First_Time_Support_Contacted: string (nullable = true)
 |-- Created_Date: date (nullable = true)
 |-- Case_Owner: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Support_Tier: string (nullable = true)
 |-- City_state: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Billing_Country: string (nullable = true)



In [0]:
# Transformations
from pyspark.sql.functions import when

# Replace "United States" with "USA" in the Country column
df = df.withColumn("Country", when(df.Country == "United States", "USA").otherwise(df.Country))

# Replace "Oui" with "Yes" and "Non" with "No" in the Issue_Resolved column
df = df.withColumn("Issue_Resolved", when(df.Issue_Resolved == "Oui", "Yes").otherwise(df.Issue_Resolved))
df = df.withColumn("Issue_Resolved", when(df.Issue_Resolved == "Non", "No").otherwise(df.Issue_Resolved))

# Display the updated DataFrame
display(df)

Client_ID,Main_Product,Number_Users,Case_Number,Overall_Technician_Satisfaction,Recommend_Friend_Colleague,Issue_Resolved,First_Time_Support_Contacted,Created_Date,Case_Owner,Category,Sub_Category,Support_Tier,City_state,Country,Billing_Country
FARME12,Epic Cloud,29,3841693,4,4,No,Yes,2022-01-02,Angelica Garcia,Epic,Download & RT Interface,Gold Advanced,"Denver, CO",USA,United States
ONECA01,Epic Cloud,39,3973271,8,8,Yes,Yes,2022-03-11,Jamar Carpenter,TAM,General,Gold Advanced,"Cedarhurst, NY",USA,United States
EMERA02,TAM Cloud,5,3846734,10,10,Yes,Yes,2022-01-03,Joshua Gralewski,TAM,General,Gold Intermediate,"Emerald Isle, NC",null,United States
NFPCA01,Epic Cloud,960,3972845,10,10,Yes,Yes,2022-03-11,David Osmolski,Epic,General,Platinum,"Ancaster, ON",USA,Canada
AZALE01,TAM Cloud,12,3842761,10,10,Yes,Yes,2022-01-03,Jamie Mercer,TAM,General,Gold Intermediate,"Wilmington, NC",null,United States
PALL-01,Epic Cloud,55,3973280,10,10,Yes,Yes,2022-03-11,Aric Calabrese,CSR24,CSR24,Gold Advanced,"Orleans, ON",null,Canada
REEDW02,Epic Cloud,43,3846783,10,10,Yes,Yes,2022-01-03,Jessica Hale,TAM,General,Gold Advanced,"Bedford, PA",USA,United States
BRYSO02,Epic Cloud,104,3973321,9,9,Yes,Yes,2022-03-11,Tammy Martin,Epic,Accounting,Platinum,"Ajax, ON",Canada,Canada
HARBI01,Epic Cloud,24,3846791,10,10,Yes,Yes,2022-01-03,Allison Arseneau,Epic,Accounting,Gold Intermediate,"Tyrone, GA",USA,United States
GRANI04,Epic Cloud,33,3973363,10,10,Yes,Yes,2022-03-11,Shareem Mohamed,Epic,Download & RT Interface,Gold Intermediate,"Granite Falls, NC",null,United States


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

**Overall Technician Satisfaction over Time:**

The line chart illustrates the trend of Overall Technician Satisfaction from January 2, 2022 to March 25, 2022. The daily average score started at 4, rising sharply to 9.9 the next day and remaining above 9.5 for most of the period. Notable peaks occurred on January 8, March 12, 19, 20 and 25, where satisfaction reached 10. The score remained stable at approximately 9.8 between January 13 and March 11. 

**Average Satisfaction by Country:**

The regional analysis shows that Canada has the lowest average satisfaction rating of 9.4, followed by the USA, i.e., 9.75. The UK has the highest average satisfaction rating of 10.

This suggests that additional focus is needed in Canada and the USA to improve satisfaction levels.

**Issue Resolution Impact:**

**Resolved Issues:** Median satisfaction score is 10, with most clients showing high satisfaction and some outliers.

**Unresolved Issues:** Median satisfaction score is 4.5, with a broad range from 1 to 10 but no significant outliers.

Resolved issues correlate with higher satisfaction scores, suggesting a need to improve resolution rates.


In [0]:
df.createOrReplaceTempView("temp_view") # create a temporary view

In [0]:
%sql
SELECT * FROM temp_view 
LIMIT 10 

Client_ID,Main_Product,Number_Users,Case_Number,Overall_Technician_Satisfaction,Recommend_Friend_Colleague,Issue_Resolved,First_Time_Support_Contacted,Created_Date,Case_Owner,Category,Sub_Category,Support_Tier,City_state,Country,Billing_Country
FARME12,Epic Cloud,29,3841693,4,4,No,Yes,2022-01-02,Angelica Garcia,Epic,Download & RT Interface,Gold Advanced,"Denver, CO",USA,United States
ONECA01,Epic Cloud,39,3973271,8,8,Yes,Yes,2022-03-11,Jamar Carpenter,TAM,General,Gold Advanced,"Cedarhurst, NY",USA,United States
EMERA02,TAM Cloud,5,3846734,10,10,Yes,Yes,2022-01-03,Joshua Gralewski,TAM,General,Gold Intermediate,"Emerald Isle, NC",null,United States
NFPCA01,Epic Cloud,960,3972845,10,10,Yes,Yes,2022-03-11,David Osmolski,Epic,General,Platinum,"Ancaster, ON",USA,Canada
AZALE01,TAM Cloud,12,3842761,10,10,Yes,Yes,2022-01-03,Jamie Mercer,TAM,General,Gold Intermediate,"Wilmington, NC",null,United States
PALL-01,Epic Cloud,55,3973280,10,10,Yes,Yes,2022-03-11,Aric Calabrese,CSR24,CSR24,Gold Advanced,"Orleans, ON",null,Canada
REEDW02,Epic Cloud,43,3846783,10,10,Yes,Yes,2022-01-03,Jessica Hale,TAM,General,Gold Advanced,"Bedford, PA",USA,United States
BRYSO02,Epic Cloud,104,3973321,9,9,Yes,Yes,2022-03-11,Tammy Martin,Epic,Accounting,Platinum,"Ajax, ON",Canada,Canada
HARBI01,Epic Cloud,24,3846791,10,10,Yes,Yes,2022-01-03,Allison Arseneau,Epic,Accounting,Gold Intermediate,"Tyrone, GA",USA,United States
GRANI04,Epic Cloud,33,3973363,10,10,Yes,Yes,2022-03-11,Shareem Mohamed,Epic,Download & RT Interface,Gold Intermediate,"Granite Falls, NC",null,United States


In [0]:
%sql
Select Case_Owner, Avg_Satis from
(Select Case_Owner,
Avg(Overall_Technician_Satisfaction) as Avg_Satis 
from temp_view 
Group By Case_Owner)
Order by Avg_Satis desc 
limit 5


Case_Owner,Avg_Satis
Cynthia Kristufek,10.0
Sam Summerson,10.0
Antonina Pemberton,10.0
Johan Schafer,10.0
Megan Patton,10.0


Databricks visualization. Run in Databricks to view.

**Case Owner Analysis:**

The top 5 case owners all have an average satisfaction score of 10:

- Cynthia Kristufek
- Sam Summerson
- Antonina Pemberton
- Johan Schafer
- Megan Patton

This indicates consistent high performance among these technicians.




**Recommendations:**


**1. Based upon Regional Analysis, focus on Regions with Lower Satisfaction**

•	**Observation:** Canada (9.4) and the USA (9.75) have lower average satisfaction ratings compared to the UK (10)

•	**Recommendation:** Investigate and address specific issues in Canada and the USA through surveys or interviews. Implement targeted initiatives such as additional training, resource allocation, mentoring and process improvements tailored to these regions.

**2. Recognize and Reward Top-Performing Technicians**

•	Establish a recognition and reward program for high-performing technicians with monthly or quarterly awards, bonuses, and public recognition, which will help maintain performance and also provide motivation to others. 

**3. Provide Training and Mentorship to Technicians Handling Unresolved Issues based upon Issue Resolution Impact.**

•	**Observation:** Cases with clients with unresolved issues by technicians have a median satisfaction score of 4.5, while those with resolved issues have a median satisfaction score of 10.

**•	Recommendation:** Share best practices and success stories by implementing training and mentorship programs, where top-performing technicians mentor their peers to improve the overall satisfaction. 

